# 🚀 YOLOv12-Face - Notebook Simple

Workflow complet : Installation → Dataset → Entraînement → Export

## 1. 📦 Installation des dépendances

In [ ]:
# Installer les dépendances nécessaires
!pip install ultralytics gdown opencv-python -q

# Vérifier l'installation
import torch
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

## 2. 📥 Préparation du dataset WIDERFace

In [ ]:
# Télécharger et préparer le dataset WIDERFace
import os
from pathlib import Path

# Vérifier si le dataset existe déjà
if Path('datasets/widerface/data.yaml').exists():
    print("✅ Dataset WIDERFace déjà présent")
else:
    print("📥 Téléchargement du dataset WIDERFace...")
    !python scripts/prepare_widerface.py --output datasets/widerface

## 3. 🏋️ Entraînement du modèle

In [ ]:
# Configuration simple
MODEL = 'yolov12n.yaml'  # ou 'ultralytics/cfg/models/v12/yolov12-face.yaml'
DATA = 'datasets/widerface/data.yaml'
EPOCHS = 100  # Réduire pour tester (10-20)
BATCH_SIZE = 16  # Ajuster selon votre GPU
DEVICE = 0 if torch.cuda.is_available() else 'cpu'

print(f"⚙️ Configuration:")
print(f"   • Modèle: {MODEL}")
print(f"   • Epochs: {EPOCHS}")
print(f"   • Batch size: {BATCH_SIZE}")
print(f"   • Device: {DEVICE}")

In [ ]:
# Lancer l'entraînement
!yolo detect train model={MODEL} data={DATA} epochs={EPOCHS} batch={BATCH_SIZE} device={DEVICE} imgsz=640 project=runs/train name=yolov12-face exist_ok=True

## 4. 📊 Validation du modèle

In [ ]:
# Valider le modèle entraîné
BEST_MODEL = 'runs/train/yolov12-face/weights/best.pt'

if Path(BEST_MODEL).exists():
    print("📊 Validation du modèle...")
    !yolo detect val model={BEST_MODEL} data={DATA} device={DEVICE}
else:
    print("❌ Modèle non trouvé. Assurez-vous que l'entraînement est terminé.")

## 5. 📦 Export du modèle

In [ ]:
# Exporter en ONNX pour le déploiement
if Path(BEST_MODEL).exists():
    print("📦 Export ONNX...")
    !yolo export model={BEST_MODEL} format=onnx simplify=True
    
    print("\n📦 Export TorchScript...")
    !yolo export model={BEST_MODEL} format=torchscript
    
    print("\n✅ Export terminé!")
else:
    print("❌ Modèle non trouvé")

## 6. 🖼️ Test sur une image

In [ ]:
# Tester sur une image d'exemple
if Path(BEST_MODEL).exists():
    # Prendre une image du dataset de validation
    val_images = list(Path('datasets/widerface/images/val').glob('*.jpg'))
    
    if val_images:
        test_image = str(val_images[0])
        print(f"🖼️ Test sur: {test_image}")
        
        # Créer un dossier pour les résultats
        !mkdir -p test_results
        
        # Lancer la détection
        !yolo detect predict model={BEST_MODEL} source={test_image} save=True project=test_results name=demo exist_ok=True
        
        print("\n✅ Résultats sauvegardés dans test_results/demo/")
    else:
        print("❌ Aucune image de test trouvée")
else:
    print("❌ Modèle non trouvé")

## 7. 📈 Visualiser les résultats

In [ ]:
# Afficher les courbes d'entraînement
from IPython.display import Image, display
import os

results_path = Path('runs/train/yolov12-face')

# Afficher les graphiques s'ils existent
plots = ['results.png', 'confusion_matrix.png', 'F1_curve.png', 'PR_curve.png']

for plot in plots:
    plot_path = results_path / plot
    if plot_path.exists():
        print(f"\n📊 {plot}:")
        display(Image(filename=str(plot_path), width=800))
    else:
        print(f"❌ {plot} non trouvé")

## 💡 Commandes utiles

In [ ]:
# Afficher les commandes utiles
print("📋 Commandes utiles pour la suite:\n")

print("# Reprendre l'entraînement:")
print(f"yolo detect train model={BEST_MODEL} data={DATA} epochs=50 resume=True\n")

print("# Tester sur une vidéo:")
print(f"yolo detect predict model={BEST_MODEL} source=video.mp4 save=True\n")

print("# Tester avec la webcam:")
print(f"yolo detect predict model={BEST_MODEL} source=0 show=True\n")

print("# Benchmark de vitesse:")
print(f"yolo benchmark model={BEST_MODEL} imgsz=640\n")

print("# Entraîner avec le script Python:")
print("python scripts/train_yolov12_face.py --model yolov12-face.yaml --epochs 100")